## Global Population Growth

In [ ]:
# Run this cell to set up the notebook, but please don't change it.

# These lines import the Numpy and Datascience modules.
import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# These lines load the tests.
from client.api.notebook import Notebook
ok = Notebook('lab11.ok')

The global population of humans reached 1 billion around 1800, 3 billion around 1960, and 7 billion around 2011. (Estimates before 1800, pictured below, are sparse.)  The potential impact of exponential population growth has concerned scientists, economists, and politicians alike.

**Question 1.** Load the table `old_pop.csv`, which contains population data back to the year 1 CE.  Make a scatter or line plot of the population over time.

In [ ]:
old_pop = Table.read_table("old_pop.csv") #SOLUTION
old_pop.scatter("Year", "Estimated Population (M)") #SOLUTION

The UN Population Division estimates that the world population will likely continue to grow throughout the 21st century, but at a slower rate, perhaps reaching 11 billion by 2100. However, the UN does not rule out scenarios of more extreme growth.

<a href="http://www.pewresearch.org/fact-tank/2015/06/08/scientists-more-worried-than-public-about-worlds-growing-population/ft_15-06-04_popcount/"> 
 <img src="pew_population_projection.png"/> 
</a>

In this lab, we will examine some of the factors that influence population growth and how they are changing around the world.

Our main dataset contains the total population of each country over time in more recent years. Run the cell below to load it.

In [ ]:
# The population.csv file can also be found online here:
# https://github.com/open-numbers/ddf--gapminder--systema_globalis/raw/master/ddf--datapoints--population_total--by--geo--time.csv
# The version here was downloaded in February, 2017.
population = Table.read_table('population.csv')
population.show(3)

### Bangladesh

In the `population` table, the `geo` column contains three-letter codes established by the [International Organization for Standardization](https://en.wikipedia.org/wiki/International_Organization_for_Standardization) (ISO) in the [Alpha-3](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3#Current_codes) standard. We will begin by taking a close look at Bangladesh. Inspect the standard to find the 3-letter code for Bangladesh.

**Question 2.** Create a table called `b_pop` that has two columns labeled `time` and `population_total`. The first column should contain the years from 1970 through 2015 (including both 1970 and 2015) and the second should contain the population of Bangladesh in each of those years.

In [ ]:
b_pop = population.where('geo', 'bgd').drop('geo').where('time', are.between(1970, 2016)) # SOLUTION
b_pop

In [ ]:
_ = ok.grade('q2')

Run the following cell to create a table called `b_five` that has the population of Bangladesh every five years. At a glance, it appears that the population of Bangladesh has been growing quickly indeed!

In [ ]:
b_pop.set_format('population_total', NumberFormatter)

fives = np.arange(1970, 2016, 5) # 1970, 1975, 1980, ...
b_five = b_pop.sort('time').where('time', are.contained_in(fives))
b_five

**Question 3.** Create a table called `b_five_growth` that includes three columns, `time`, `population_total`, and `annual_growth`. There should be one row for every five years from 1970 through 2010 (but not 2015). The first two columns are the same as `b_five`. The third column is the **annual** growth rate for each five-year period. For example, the annual growth rate for 1975 is the yearly exponential growth rate that describes the total growth from 1975 to 1980 when applied 5 times.

*Hint*: Only your `b_five_growth` table is scored for correctness; the other names are suggestions that you are welcome to use, change, or delete.

In [ ]:
b_1970_through_2010 = b_five.where('time', are.below_or_equal_to(2010)) # SOLUTION
initial = b_1970_through_2010.column(1) # SOLUTION
changed = b_five.where('time', are.above_or_equal_to(1975)).column(1) # SOLUTION
b_five_growth = b_1970_through_2010.with_column('annual_growth', (changed/initial)**0.2-1) # SOLUTION
b_five_growth

In [ ]:
_ = ok.grade('q3')

While the population has grown every five years since 1970, the annual growth rate decreased dramatically from 1985 to 2005. Let's look at some other information in order to develop a possible explanation. Run the next cell to load three additional tables of measurements about countries over time.

In [ ]:
life_expectancy = Table.read_table('life_expectancy.csv')
child_mortality = Table.read_table('child_mortality.csv').relabeled(2, 'child_mortality_under_5_per_1000_born')
fertility = Table.read_table('fertility.csv')

The `life_expectancy` table contains a statistic that is often used to measure how long people live, called *life expectancy at birth*. This number, for a country in a given year, [does not measure how long babies born in that year are expected to live](http://blogs.worldbank.org/opendata/what-does-life-expectancy-birth-really-mean). Instead, it measures how long someone would live, on average, if the *mortality conditions* in that year persisted throughout their lifetime. These "mortality conditions" describe what fraction of people at each age survived the year. So, it is a way of measuring the proportion of people that are staying alive, aggregated over different age groups in the population.

**Question 4.** Perhaps population is growing more slowly because people aren't living as long. Use the `life_expectancy` table to draw a line graph with the years 1970 and later on the horizontal axis that shows how the *life expectancy at birth* has changed in Bangladesh.

In [ ]:
life_expectancy.where('geo', 'bgd').where('time', are.above(1969)).relabeled('time', 'Year').plot(1, 2) #SOLUTION

**Question 5.** Does the graph above help directly explain why the population growth rate decreased from 1985 to 2010 in Bangladesh? Why or why not? What happened in Bangladesh in 1991, and does that event explain the change in population growth rate?

**SOLUTION:** This graph indicates that people are living longer, which would increase population growth if everything else stayed the same. The tragic cyclone in 1991 certainly affected population size, but life expectancy contnued to increase shortly afterward, so it does not explain the 25-year trend in population growth rate decline.

The `fertility` table contains a statistic that is often used to measure how many babies are being born, the *total fertility rate*. This number describes the [number of children a woman would have in her lifetime](https://www.measureevaluation.org/prh/rh_indicators/specific/fertility/total-fertility-rate), on average, if the current rates of birth by age of the mother persisted throughout her child bearing years, assuming she survived through age 49. 

**Question 6.** Write a function `fertility_over_time` that takes the Alpha-3 code of a `country` and a `start` year. It returns a two-column table with labels "`Year`" and "`Children per woman`" that can be used to generate a line chart of the country's fertility rate each year, starting at the `start` year. The plot should include the `start` year and all later years that appear in the `fertility` table. 

Then, in the next cell, call your `fertility_over_time` function on the Alpha-3 code for Bangladesh and the year 1970 in order to plot how Bangladesh's fertility rate has changed since 1970. **The expression that draws the line plot is provided for you; please don't change it.**

In [ ]:
def fertility_over_time(country, start):
    """Create a two-column table that describes a country's total fertility rate each year."""
    return fertility.where('geo', are.equal_to(country)).where('time', are.above_or_equal_to(start)).select(1, 2).relabeled(0, 'Year').relabeled(1, 'Children per woman') # SOLUTION

In [ ]:
bangladesh_code = 'bgd' # SOLUTION
fertility_over_time(bangladesh_code, 1970).plot(0, 1) # You should *not* change this line.

In [ ]:
_ = ok.grade('q6')

**Question 7.** Does the graph above help directly explain why the population growth rate decreased from 1985 to 2010 in Bangladesh? Why or why not?

**SOLUTION:** Yes, a declining fertility rate shows that fewer babies are being born each year, which directly explains decreasing population growth.

It has been observed that lower fertility rates are often associated with lower child mortality rates. The link has been attributed to family planning: if parents can expect that their children will all survive into adulthood, then they will choose to have fewer children. We can see if this association is evident in Bangladesh by plotting the relationship between total fertility rate and [child mortality rate per 1000 children](https://en.wikipedia.org/wiki/Child_mortality).

The cell below joins the `fertility` and `child_mortality` tables together into a single table, to facilitate comparisons.  Run it and read the output.

In [ ]:
fertility_and_child_mortality = fertility.where('geo', 'bgd').drop('geo').join('time', child_mortality.where('geo', 'bgd').drop('geo')).relabeled(1, 'Children per woman').relabeled(2, 'Child deaths per 1000 born')
fertility_and_child_mortality

**Question 8.** Using both the `fertility_and_child_mortality` table, draw a scatter diagram with one point for each year, **starting with 1970**, that has Bangladesh's total fertility on the horizontal axis and its child mortality on the vertical axis.

In [ ]:
fertility_and_child_mortality.where('time', are.above(1969)).scatter('Children per woman', 'Child deaths per 1000 born') #SOLUTION

**Question 9.** In one or two sentences, describe the association (if any) that is illustrated by this scatter diagram. Does the diagram show that reduced child mortality causes parents to choose to have fewer children?

**SOLUTION:** We can observe a very strong linear association between fertility rate and child mortality rate. It is strong because the points fall so near to a line drawn through the diagram. However, this association does not tell us whether one of these changes caused a change in the other.

### The World (optional review of histograms)

The change observed in Bangladesh since 1970 can also be observed in many other developing countries: health services improve, life expectancy increases, and child mortality decreases. At the same time, the fertility rate often plummets, and so the population growth rate decreases despite increasing longevity.

Run the cell below to generate two overlaid histograms, one for 1960 and one for 2010, that show the distributions of total fertility rates for these two years among all 201 countries in the `fertility` table.

In [ ]:
Table().with_columns(
    '1960', fertility.where('time', 1960).column(2),
    '2010', fertility.where('time', 2010).column(2)
).hist(bins=np.arange(0, 10, 0.5), unit='child')
_ = plots.xlabel('Children per woman')
_ = plots.xticks(np.arange(10))

**Question 10.** Assign `fertility_statements` to an array of the numbers for each statement below that can be correctly inferred from these histograms.
1. About the same number of countries had a fertility rate between 3.5 and 4.5 in both 1960 and 2010.
1. In 2010, about 40% of countries had a fertility rate between 1.5 and 2.
1. In 1960, less than 20% of countries had a fertility rate below 3.
1. More countries had a fertility rate above 3 in 1960 than in 2010.
1. At least half of countries had a fertility rate between 5 and 8 in 1960.
1. At least half of countries had a fertility rate below 3 in 2010.

In [ ]:
fertility_statements = make_array(1, 3, 4, 5, 6) # SOLUTION

In [ ]:
_ = ok.grade('q10')

If you still have time, take a break and watch this spirited presentation by [Hans Rosling in a 2010 TEDx talk](https://www.gapminder.org/videos/reducing-child-mortality-a-moral-and-environmental-imperative) on mortality and fertility in the world.

In [ ]:
_ = ok.submit()